In [1]:
from importlib import reload
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from transformers import (
    AutoConfig, AutoModelForCausalLM, AutoTokenizer
)

pl.seed_everything(42)

/home/soroosh/projects/general_env/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(
Global seed set to 42


42

## tokenizer and dataset

In [2]:
name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(name)

tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>',
})

0

In [3]:
import src.data
reload(src.data)
from src.data import PoemDataset, get_dataloaders

dataset = PoemDataset(tokenizer, 'data/all_poems.json', max_len=512)
len(dataset)

11579631

In [6]:
tl, vl = get_dataloaders(dataset, val_frac=0.1, batch_size=4)

train dataset has 10421668 samples and val dataset has 1157963 samples


In [7]:
b = next(iter(tl))
for k in b:
    print(k, b[k].shape)

input_ids torch.Size([4, 167])
attention_mask torch.Size([4, 167])


## model

In [8]:
import src.model
reload(src.model)
from src.model import PoetFormer

name = 'GPT2-fa-ganjoor-conditional'
print('model name:',name)

# model = PoetFormer(pretrained="HooshvareLab/gpt2-fa")
model = PoetFormer.load_from_checkpoint(f'weights/{name}/last.ckpt', pretrained="HooshvareLab/gpt2-fa")

model name: GPT2-fa-ganjoor-conditional
using pretrained model: HooshvareLab/gpt2-fa


In [7]:
model.count_parameters()

118099200

In [8]:
res = model.generate(num_return_sequences=1, max_length=128, n_beam=1)
for r in res:
    print(r)

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


حافظ:
وی «ان» هم که از این کلمه‌ها در ادبیات ما استفاده شده، این کلمه‌ها، به شکل «ان» هستند. حال، یک سوال مطرح می‌شود که چطور می‌توانیم این کلمات را در کلماتمان جای دهیم؟ این سوال، یکی از مسائل بسیار مهم در علم منطق است. «ان» در لغت به معنی «بودن» است و در واقع همان «بودن» است. «ان» در زبان فارسی به معنای «بودن» است و «بودن» به معنی «بودن» است. پس «بودن» یعنی «بودن». برای پاسخ به این سوال، فرض کنید می‌


## train

In [9]:
logger = TensorBoardLogger(save_dir='logs/', name=name)
lr_logger = LearningRateMonitor(logging_interval='step')
checkpoint = ModelCheckpoint(
    dirpath=f'weights/{name}/', 
    filename='{epoch}-{val_loss:.2f}', 
    monitor='val_loss',
    save_top_k=1, 
    period=1
)

trainer = pl.Trainer(
    benchmark=True, 
    gpus=1, 
    accumulate_grad_batches=1,
    logger=logger, 
    max_epochs=1,
    callbacks=[checkpoint, lr_logger]
)

/home/soroosh/projects/general_env/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:396: LightningDeprecationWarning: Argument `period` in `ModelCheckpoint` is deprecated in v1.3 and will be removed in v1.5. Please use `every_n_val_epochs` instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


model name: GPT2-fa-ganjoor-conditional


In [10]:
trainer.fit(model, tl, vl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 118 M 
------------------------------------------
118 M     Trainable params
0         Non-trainable params
118 M     Total params
472.397   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 426.00 MiB (GPU 0; 10.76 GiB total capacity; 8.42 GiB already allocated; 427.81 MiB free; 9.26 GiB reserved in total by PyTorch)

In [15]:
trainer.save_checkpoint(f'weights/{name}/last.ckpt')

## generate

In [12]:
model.cpu()

PoetFormer(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(42001, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            

In [14]:
res = model.generate(prompt='ای پادشه خوبان', num_return_sequences=1, max_length=128, n_beam=1)
for r in res:
    print(r.replace('<sep>', '\n'))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


حافظ:
ای پادشه خوبان که در ملک توران
در دل من است این از پی آن از پی آن از پی آن
ای شیر خداوندی که در عالم ملک
چون من گدا از پی آن باشدت این از پی آن
چونان که بر خوان فقر و درویشی نانش
جز بر خوان فقر و درویشی نانش باشد نانش
ای بی‌خبر از پی نان تا در عالم جان
ننگ باشد تو را در این بی‌حاصلی نانش
تا به عالم دون بود کی نانشان بود
تا به این دوران بود آن نانشان نانشان نانش
تا به دوران بود آن نان
